# Nå skal vi løse et Traveling Salesperson Problem med OR-tools

Gjennomgangen er basert på dette problemet: https://developers.google.com/optimization/routing/tsp


### Google OR-Tools kan brukes til mye forskjellig
Vi skal bruke det til å løse et Traveling Salesperson Problem, men det kan også brukes på f.eks Vehicle Routing Problem (TSP med flere biler), kapasitetrestriksjoner, tidsvidu, ressursrestriksjoner osv.

Vi går gjennom koden sammen.

python -m pip install --upgrade --user ortools

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

### Lage data

Vi starter med å lage data - en distansematrise for entry (i,j) er distansen mellom lokasjon i og j i miles. 

Så på kartet under for å se hvor vi skal finne en best mulig rute. Lokasjonene vi skal se på er: 0. New York - 1. Los Angeles - 2. Chicago - 3. Minneapolis - 4. Denver - 5. Dallas - 6. Seattle - 7. Boston - 8. San Francisco - 9. St. Louis - 10. Houston - 11. Phoenix - 12. Salt Lake City.


Vi må også spesifisere antall biler, i tillegg til start-og sluttlokasjon for ruten. 

In [ ]:
data = {}
data['distance_matrix'] = [
    [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
    [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
    [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
    [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
    [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
    [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
    [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
    [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
    [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
    [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
    [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
    [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
    [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
]
data['num_vehicles'] = 2
data['depot'] = 0

![title](kart_tsp.png)

### RoutingIndexManager

Må spesifisere: 
* antall lokasjoner (antall rader i distansematrisen)
* antall biler i problemet
* start-og sluttlokasjon



In [ ]:
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

### Lag routing model

In [ ]:
routing = pywrapcp.RoutingModel(manager)

### Lag distance 
En funksjon som tar par av lokasjoner (from_index, to_index) og returnerer avstanden mellom lokasjonene.

manager.IndexToNode konverterer mellom solverns "interne indexer" og indexene vi har spesifisert for lokasjonene.

In [ ]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

### Definer kostnaden ved å reise med arc cost evaluator
Her er kostnaden ved å reise mellom to lokasjoner avstanden mellom. Her er det mulig å inkludere andre faktorer også.

In [ ]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

### Bestem søk-parametre og heuristikk-metoden for å finne den første løsningen

Vi bruker default søk-parametre.

routing_enums_pb2: search strategy vi benytter i dette eksempelet.

PATH_CHEAPEST_ARC: lager en initiell rute for solveren ved å legge til kanter med lavest kostand som ikke fører til en tidligere besøkt node.

In [ ]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

### Print løsningen

In [ ]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

In [ ]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

![title](tsp_rute.jpg)

### Hvis tid - endre search strategy

In [ ]:
search_parameters_2 = pywrapcp.DefaultRoutingSearchParameters()
search_parameters_2.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
#search_parameters.time_limit.seconds = 30
#search_parameters.log_search = True

solution = routing.SolveWithParameters(search_parameters_2)
if solution:
    print_solution(manager, routing, solution)